# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user"s session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user"s browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
!pip install pyspark==3.2.1

In [2]:
# create Spark session

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(appName="sga").getOrCreate()

from pyspark.sql import SparkSession
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-27 12:35:53,866 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
# data loading

clickstream_dir = "hdfs:/data/clickstream.csv"

clickstream_df = se.read.csv(clickstream_dir, sep="\t", header=True)
clickstream_df.show(truncate=False)

+-------+----------+------------+----------+----------+
|user_id|session_id|event_type  |event_page|timestamp |
+-------+----------+------------+----------+----------+
|562    |507       |page        |main      |1695584127|
|562    |507       |event       |main      |1695584134|
|562    |507       |event       |main      |1695584144|
|562    |507       |event       |main      |1695584147|
|562    |507       |wNaxLlerrorU|main      |1695584154|
|562    |507       |event       |main      |1695584154|
|562    |507       |event       |main      |1695584154|
|562    |507       |event       |main      |1695584160|
|562    |507       |page        |rabota    |1695584166|
|562    |507       |event       |rabota    |1695584174|
|562    |507       |event       |rabota    |1695584181|
|562    |507       |event       |rabota    |1695584189|
|562    |507       |page        |main      |1695584194|
|562    |507       |event       |main      |1695584204|
|562    |507       |event       |main      |1695

In [4]:
clickstream_df.createOrReplaceTempView("clickstream_df")

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

In [6]:
def generate_route(events: list[tuple[str, str, str]]) -> str:
    '''
    Generates the route for user session
    '''
    sorted_events = sorted(events, key=lambda x: x["timestamp"])
    route = []
    last_page = None
    
    for event in sorted_events:
        # stop process session after error
        if "error" in event["event_type"]:
            break
        
        # check page change
        if event["event_type"] == "page" and event["event_page"] != last_page:
            route.append(event["event_page"])
            last_page = event["event_page"]

    return "-".join(route)

## Spark SQL

In [7]:
se.udf.register("generate_route_udf", generate_route, StringType())

<function __main__.generate_route(events: list[tuple[str, str, str]]) -> str>

In [8]:
# generate all possible routes

generated_routes_sql = se.sql("""
    SELECT 
        user_id,
        session_id,
        generate_route_udf(collect_list(struct(event_type, event_page, timestamp))) AS route
    FROM
        clickstream_df
    GROUP BY 
        user_id, session_id
""")

generated_routes_sql.show()

+-------+----------+--------------------+
|user_id|session_id|               route|
+-------+----------+--------------------+
|      1|      1026|                main|
|     10|       762|main-internet-arc...|
|   1002|       285|main-news-interne...|
|   1007|        82|main-archive-digital|
|   1009|       639|main-online-inter...|
|   1010|       141|main-internet-arc...|
|   1014|       341|main-internet-tar...|
|   1019|       669|main-bonus-main-a...|
|   1029|        58|main-archive-digital|
|    103|        72|main-bonus-archiv...|
|    103|        81|         main-rabota|
|   1030|       890|main-archive-bonu...|
|   1032|       257|          main-vklad|
|   1033|       604|   main-bonus-rabota|
|    104|       185|main-rabota-onlin...|
|   1040|       393| main-vklad-internet|
|   1040|       415|main-tariffs-arch...|
|   1042|       950|main-news-rabota-...|
|   1045|        12|          main-vklad|
|   1048|       252|main-archive-main...|
+-------+----------+--------------

In [9]:
generated_routes_sql.createOrReplaceTempView("generated_routes_sql")

In [10]:
# count unique routes and take top-30 

top_routes_sql = se.sql("""
    SELECT 
        route, 
        COUNT(*) AS count
    FROM 
        generated_routes_sql
    GROUP BY
        route
    ORDER BY 
        count DESC
    LIMIT 30
""")

top_routes_sql.show(truncate=False)

+---------------------+-----+
|route                |count|
+---------------------+-----+
|main                 |8185 |
|main-archive         |1113 |
|main-rabota          |1047 |
|main-internet        |897  |
|main-bonus           |870  |
|main-news            |769  |
|main-tariffs         |677  |
|main-online          |587  |
|main-vklad           |518  |
|main-rabota-archive  |170  |
|main-archive-rabota  |167  |
|main-bonus-archive   |143  |
|main-rabota-bonus    |139  |
|main-bonus-rabota    |135  |
|main-news-rabota     |135  |
|main-archive-internet|132  |
|main-rabota-news     |130  |
|main-internet-rabota |129  |
|main-archive-news    |126  |
|main-rabota-internet |124  |
+---------------------+-----+
only showing top 20 rows



In [11]:
# write solution file

top_routes_sql \
    .coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("sep", "\t") \
    .csv("hdfs:/data/top_routes_sql")

!rm -f ./top_routes_sql.txt
!hdfs dfs -ls hdfs:/data/top_routes_sql
!hdfs dfs -get hdfs:/data/top_routes_sql/part-00000-* ./top_routes_sql.txt

Found 2 items
-rw-r--r--   1 jovyan supergroup          0 2024-10-27 12:37 hdfs:///data/top_routes_sql/_SUCCESS
-rw-r--r--   1 jovyan supergroup        632 2024-10-27 12:37 hdfs:///data/top_routes_sql/part-00000-5378b071-82ed-4ec9-8193-87719d5cecb4-c000.csv


In [12]:
top_10_routes_sql = top_routes_sql.limit(10).collect()
top_10_routes_sql_list = {row['route']: row['count'] for row in top_10_routes_sql}

top_10_routes_sql_list

{'main': 8185,
 'main-archive': 1113,
 'main-rabota': 1047,
 'main-internet': 897,
 'main-bonus': 870,
 'main-news': 769,
 'main-tariffs': 677,
 'main-online': 587,
 'main-vklad': 518,
 'main-rabota-archive': 170}

## Spark RDD

In [13]:
clickstream_rdd = clickstream_df.rdd

In [14]:
parsed_rdd = clickstream_rdd.map(lambda row: {
    "user_id": int(row[0]),
    "session_id": int(row[1]),
    "timestamp": int(row[4]),
    "event_type": row[2],
    "event_page": row[3]
})

In [15]:
# generate all possible routes

generated_routes_rdd = parsed_rdd \
    .groupBy(lambda x: (x["user_id"], x["session_id"])) \
    .mapValues(generate_route)

In [16]:
# count unique routes and take top-30 

top_routes_rdd = generated_routes_rdd \
    .map(lambda x: (x[1], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .top(30, key=lambda x: x[1])

top_routes_rdd

[('main', 8185),
 ('main-archive', 1113),
 ('main-rabota', 1047),
 ('main-internet', 897),
 ('main-bonus', 870),
 ('main-news', 769),
 ('main-tariffs', 677),
 ('main-online', 587),
 ('main-vklad', 518),
 ('main-rabota-archive', 170),
 ('main-archive-rabota', 167),
 ('main-bonus-archive', 143),
 ('main-rabota-bonus', 139),
 ('main-news-rabota', 135),
 ('main-bonus-rabota', 135),
 ('main-archive-internet', 132),
 ('main-rabota-news', 130),
 ('main-internet-rabota', 129),
 ('main-archive-news', 126),
 ('main-rabota-internet', 124),
 ('main-internet-archive', 123),
 ('main-archive-bonus', 117),
 ('main-internet-bonus', 115),
 ('main-news-archive', 113),
 ('main-tariffs-internet', 113),
 ('main-news-internet', 109),
 ('main-archive-tariffs', 104),
 ('main-internet-news', 103),
 ('main-tariffs-archive', 103),
 ('main-rabota-main', 94)]

In [17]:
# write solution file

!rm -f ./top_routes_rdd.txt

with open('top_routes_rdd.txt', "w") as f:
    for route, count in top_routes_rdd:
        f.write(f"{route}\t{count}\n")

In [18]:
top_10_routes_rdd_list = top_routes_rdd[:10]

top_10_routes_rdd_list

[('main', 8185),
 ('main-archive', 1113),
 ('main-rabota', 1047),
 ('main-internet', 897),
 ('main-bonus', 870),
 ('main-news', 769),
 ('main-tariffs', 677),
 ('main-online', 587),
 ('main-vklad', 518),
 ('main-rabota-archive', 170)]

## Spark DF

In [19]:
generate_route_udf = F.udf(lambda events: generate_route(events), StringType())

In [20]:
# generate all possible routes

generated_routes_df = clickstream_df \
    .groupBy("user_id", "session_id") \
    .agg(F.collect_list(F.struct("event_type", "event_page", "timestamp")) \
    .alias("events")) \
    .withColumn("route", generate_route_udf(F.col("events"))) \
    .drop("events")

In [21]:
# count unique routes and take top-30 

top_routes_df = generated_routes_df \
    .groupBy("route") \
    .count() \
    .orderBy(F.col("count").desc()) \
    .limit(30)

top_routes_df.collect()

[Row(route='main', count=8185),
 Row(route='main-archive', count=1113),
 Row(route='main-rabota', count=1047),
 Row(route='main-internet', count=897),
 Row(route='main-bonus', count=870),
 Row(route='main-news', count=769),
 Row(route='main-tariffs', count=677),
 Row(route='main-online', count=587),
 Row(route='main-vklad', count=518),
 Row(route='main-rabota-archive', count=170),
 Row(route='main-archive-rabota', count=167),
 Row(route='main-bonus-archive', count=143),
 Row(route='main-rabota-bonus', count=139),
 Row(route='main-news-rabota', count=135),
 Row(route='main-bonus-rabota', count=135),
 Row(route='main-archive-internet', count=132),
 Row(route='main-rabota-news', count=130),
 Row(route='main-internet-rabota', count=129),
 Row(route='main-archive-news', count=126),
 Row(route='main-rabota-internet', count=124),
 Row(route='main-internet-archive', count=123),
 Row(route='main-archive-bonus', count=117),
 Row(route='main-internet-bonus', count=115),
 Row(route='main-tariffs-i

In [22]:
# write solution file

top_routes_df \
    .coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("sep", "\t") \
    .csv("hdfs:/data/top_routes_df")

!rm -f ./top_routes_df.txt
!hdfs dfs -ls hdfs:/data/top_routes_df
!hdfs dfs -get hdfs:/data/top_routes_df/part-00000-* ./top_routes_df.txt

Found 2 items
-rw-r--r--   1 jovyan supergroup          0 2024-10-27 12:38 hdfs:///data/top_routes_df/_SUCCESS
-rw-r--r--   1 jovyan supergroup        632 2024-10-27 12:38 hdfs:///data/top_routes_df/part-00000-a386d9b7-ae0c-4a7e-8687-0724617a0b30-c000.csv


In [23]:
top_10_routes_df = top_routes_df.limit(10).collect()
top_10_routes_df_list = {row['route']: row['count'] for row in top_10_routes_df}

top_10_routes_df_list

{'main': 8185,
 'main-archive': 1113,
 'main-rabota': 1047,
 'main-internet': 897,
 'main-bonus': 870,
 'main-news': 769,
 'main-tariffs': 677,
 'main-online': 587,
 'main-vklad': 518,
 'main-rabota-archive': 170}

In [24]:
# close Spark session

se.stop()
sc.stop()

### Grader check

In [25]:
import json

with open('result.json', 'w') as f:
    json.dump(top_10_routes_df_list, f)

In [26]:
# !curl -F file=@result.json 51.250.123.136:80/MDS-LSML1/pyatykhina/w6/2

0.9999999999999999
Correct main answer!
Correct main-archive answer!
Correct main-rabota answer!
Correct main-internet answer!
Correct main-bonus answer!
Correct main-news answer!
Correct main-tariffs answer!
Correct main-online answer!
Correct main-vklad answer!
Correct main-rabota-archive answer!
